In [8]:
# Import packages and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest

In [9]:
# Import dataset
cancer = pd.read_csv('../data/raw/breast.csv')

# Changing the names of the columns to improve their understanding
new_columns = ['ID', 'Diagnosis']
for i in range (30):       
       if cancer.columns[i+2][-1:] == '1': 
               column_type = 'mean'
       elif cancer.columns[i+2][-1:] == '2': 
               column_type = 'se'
       else: 
               column_type = 'worst'
       temp = cancer.columns[i+2][:-1] + '_' + column_type
       new_columns.append(temp)

cancer.columns = new_columns
cancer.describe()

,ID,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [10]:
# Standardize features using the z-score method
scaler = StandardScaler()

features = cancer.columns[2:]

standardized_cancer = scaler.fit_transform(cancer[features])

# Convert the standardized data back to a DataFrame
standardized_cancer = pd.DataFrame(standardized_cancer, columns=features)

standardized_cancer.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,...,5.690000e+02,5.690000e+02,5.690000e+02,569.000000,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02
mean,-1.373633e-16,6.868164e-17,-1.248757e-16,-2.185325e-16,-8.366672e-16,1.873136e-16,4.995028e-17,-4.995028e-17,1.748260e-16,4.745277e-16,...,-8.241796e-16,1.248757e-17,-3.746271e-16,0.000000,-2.372638e-16,-3.371644e-16,7.492542e-17,2.247763e-16,2.622390e-16,-5.744282e-16
std,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,...,1.000880e+00,1.000880e+00,1.000880e+00,1.000880,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00
min,-2.029648e+00,-2.229249e+00,-1.984504e+00,-1.454443e+00,-3.112085e+00,-1.610136e+00,-1.114873e+00,-1.261820e+00,-2.744117e+00,-1.819865e+00,...,-1.726901e+00,-2.223994e+00,-1.693361e+00,-1.222423,-2.682695e+00,-1.443878e+00,-1.305831e+00,-1.745063e+00,-2.160960e+00,-1.601839e+00
25%,-6.893853e-01,-7.259631e-01,-6.919555e-01,-6.671955e-01,-7.109628e-01,-7.470860e-01,-7.437479e-01,-7.379438e-01,-7.032397e-01,-7.226392e-01,...,-6.749213e-01,-7.486293e-01,-6.895783e-01,-0.642136,-6.912304e-01,-6.810833e-01,-7.565142e-01,-7.563999e-01,-6.418637e-01,-6.919118e-01
50%,-2.150816e-01,-1.046362e-01,-2.359800e-01,-2.951869e-01,-3.489108e-02,-2.219405e-01,-3.422399e-01,-3.977212e-01,-7.162650e-02,-1.782793e-01,...,-2.690395e-01,-4.351564e-02,-2.859802e-01,-0.341181,-4.684277e-02,-2.695009e-01,-2.182321e-01,-2.234689e-01,-1.274095e-01,-2.164441e-01
75%,4.693926e-01,5.841756e-01,4.996769e-01,3.635073e-01,6.361990e-01,4.938569e-01,5.260619e-01,6.469351e-01,5.307792e-01,4.709834e-01,...,5.220158e-01,6.583411e-01,5.402790e-01,0.357589,5.975448e-01,5.396688e-01,5.311411e-01,7.125100e-01,4.501382e-01,4.507624e-01
max,3.971288e+00,4.651889e+00,3.976130e+00,5.250529e+00,4.770911e+00,4.568425e+00,4.243589e+00,3.927930e+00,4.484751e+00,4.910919e+00,...,4.094189e+00,3.885905e+00,4.287337e+00,5.930172,3.955374e+00,5.112877e+00,4.700669e+00,2.685877e+00,6.046041e+00,6.846856e+00


In [11]:
# Split the dataset into test and training sets at 25-75 split.

X_train, X_test, Y_train, Y_test = train_test_split(standardized_cancer, 
                                                    cancer['Diagnosis'], 
                                                    test_size=0.25, random_state=42)

# Verify shapes
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(426, 30) (143, 30)
(426,) (143,)


In [18]:
# As part of our analysis we will need a metric to decide which feature to select in case other metrics (e.g. accuracy produces equal results)
# Therefore, we suggest to use Chi-square test to select features based on their statistical significance relative to the target variable

le = LabelEncoder()
# Apply Label Encoding to each column in your dataset if necessary
for column in standardized_cancer.columns:
    standardized_cancer[column] = le.fit_transform(standardized_cancer[column])

# Assuming 'X' contains your features and 'y' is your target variable
chi2_selector = SelectKBest(chi2, k='all')  # k='all' to compute scores for all features
chi2_selector.fit(standardized_cancer, cancer['Diagnosis'])

# Get Chi-square scores and p-values
chi2_scores = chi2_selector.scores_
p_values = chi2_selector.pvalues_

# Creating a DataFrame to summarize scores and p-values
chi2_results = pd.DataFrame({"feature": standardized_cancer.columns, "Chi2_Score": chi2_scores, "P_Value": p_values})
# Sort features by score or p-value
# significant_features = chi2_results[chi2_results["P_Value"] < 0.05].sort_values(by="Chi2_Score", ascending=False)

# print(significant_features)
chi2_results['chi2_rank'] = chi2_results['Chi2_Score'].rank(ascending=False)
chi2_results = chi2_results.sort_values(by='chi2_rank', ascending=True)
chi2_results

,feature,Chi2_Score,P_Value,chi2_rank
7,concave_points_mean,33181.619418,0.000000e+00,1.0
23,area_worst,31608.293011,0.000000e+00,2.0
27,concave_points_worst,30176.634384,0.000000e+00,3.0
6,concavity_mean,28964.980562,0.000000e+00,4.0
22,perimeter_worst,28744.082662,0.000000e+00,5.0
2,perimeter_mean,27327.235675,0.000000e+00,6.0
3,area_mean,27274.927380,0.000000e+00,7.0
26,concavity_worst,26496.098263,0.000000e+00,8.0
13,area_se,26300.763509,0.000000e+00,9.0
20,radius_worst,25680.903722,0.000000e+00,10.0


In [21]:
# 1) We suggest to follow the Forward Feature Selection method to make the decision on the most important features.
# 2) The general idea is to iteratively add features based on the KNN method performance (we'll be optimizing accuracy).
# 3) At the first step we'll run independently KNN for every feature to decide which 
# one of them produces KNN with the highest accuracy score (such feature will be selected as feature1).
# 4) At the second step we'll run independently KNN for feature1 and every other feature to decide which 
# combination of feature1 and another feature produces KNN with the highest accuracy score (this another feature is feature2).
# 5) We'll be doing this up to 15 features. We think it should be enough for our purpose 
# to limit the number of features at least two times as compared to the initial 30 features.
knn = KNeighborsClassifier()
parameter_grid = {
    "n_neighbors": range(1, 30),
}

cancer_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10
)

# structure of the final DataFrame with the results. The results presented in every line will be for all features above
KNN_forward_results = pd.DataFrame(columns=
                                           ['feature',
                                            'opt_n_neighbors', 
                                            'accuracy', 
                                            'precision', 
                                            'recall', 
                                            'f1']
                                           )

KNN_interim_results = pd.DataFrame(columns=
                                           ['feature', 
                                            'opt_n_neighbors', 
                                            'accuracy', 
                                            'precision', 
                                            'recall', 
                                            'f1']
                                           )

stage = 1
feature_count = 0
while feature_count < 15: 
    
    if stage == 1:
        working_features = features	# list of features to used for identifing the feature that helps to produce the highest accuracy score
    else:
        working_features = pd.Index(list(set(working_features) - set(selected_features['feature'])))
        X_train_base = X_train[KNN_forward_results['feature'].tolist()] # "base" dataset to train the models (it will be iteratively expanded, see below)
        X_test_base = X_test[KNN_forward_results['feature'].tolist()] # "base" dataset to test the models (it will be iteratively expanded, see below)
        KNN_interim_results = KNN_interim_results.drop(KNN_interim_results.index)
        selected_features = selected_features.drop(selected_features.index)    

    for i in working_features:
        if stage == 1:
            X_train_exp = X_train[[i]]
            X_test_exp = X_test[[i]]
        else:
            X_train_exp = X_train_base.join(X_train[i], how='inner')
            X_test_exp = X_test_base.join(X_test[i], how='inner')

        cancer_tune_grid.fit(
            X_train_exp,
            Y_train
        )

        # Refit the KNN model with the best k value
        knn_best = KNeighborsClassifier(n_neighbors=cancer_tune_grid.best_params_['n_neighbors'])
        knn_best.fit(X_train_exp, Y_train)

        # Predict on the test set
        
        X_pred = X_test_exp
        Y_pred = knn_best.predict(X_pred)

        new_row = pd.DataFrame({
            'feature': [i],
            'opt_n_neighbors': [cancer_tune_grid.best_params_['n_neighbors']],
            'accuracy': [accuracy_score(Y_test, Y_pred)],
            'precision': [precision_score(Y_test,Y_pred,pos_label="M")],
            'recall': [recall_score(Y_test,Y_pred,pos_label="M")],
            'f1': [f1_score(Y_test,Y_pred, average='weighted')] 
        })
        if KNN_interim_results.empty: 
            KNN_interim_results = new_row
        else:
            KNN_interim_results = pd.concat([KNN_interim_results, new_row], ignore_index=True)
    
    KNN_interim_results['accuracy_rank'] = KNN_interim_results['accuracy'].rank(ascending=False)

    KNN_interim_results = pd.merge(KNN_interim_results, chi2_results[['feature','Chi2_Score']], on='feature', how='inner')
    
    interim_selection = KNN_interim_results[KNN_interim_results['accuracy_rank'] == min(KNN_interim_results['accuracy_rank'])]
    selected_features = interim_selection[interim_selection['Chi2_Score'] == max(interim_selection['Chi2_Score'])]    
    
    for j in range(len(selected_features)): # in case there could be 2 or more lines with the same accuracy scores and Chi2 scores
        new_row = pd.DataFrame({
            'feature': [selected_features.iloc[j]['feature']],
            'opt_n_neighbors': [selected_features.iloc[j]['opt_n_neighbors']],
            'accuracy': [selected_features.iloc[j]['accuracy']],
            'precision': [selected_features.iloc[j]['precision']],
            'recall': [selected_features.iloc[j]['recall']],
            'f1': [selected_features.iloc[j]['f1']]
        })
        if KNN_forward_results.empty: 
            KNN_forward_results = new_row
        else:
            KNN_forward_results = pd.concat([KNN_forward_results, new_row], ignore_index=True)
    stage += 1
    feature_count = KNN_forward_results.shape[0]

KNN_forward_results = pd.merge(KNN_forward_results, chi2_results[['feature','Chi2_Score']], on='feature', how='inner')
KNN_forward_results

,feature,opt_n_neighbors,accuracy,precision,recall,f1,Chi2_Score
0,area_worst,16,0.951049,0.979592,0.888889,0.950530,31608.293011
1,compactness_se,17,0.979021,1.000000,0.944444,0.978895,7301.703489
2,concavity_mean,5,0.986014,0.981481,0.981481,0.986014,28964.980562
3,perimeter_worst,17,0.986014,0.981481,0.981481,0.986014,28744.082662
4,area_se,13,0.986014,0.981481,0.981481,0.986014,26300.763509
5,compactness_worst,9,0.993007,0.981818,1.000000,0.993019,18250.856307
6,concave_points_se,4,0.986014,0.981481,0.981481,0.986014,11447.885600
7,smoothness_worst,8,0.993007,1.000000,0.981481,0.992994,6043.389537
8,area_mean,5,0.993007,0.981818,1.000000,0.993019,27274.927380
9,radius_worst,5,0.993007,0.981818,1.000000,0.993019,25680.903722


In [27]:
# Based on the results in the table above, adding any features 
# beyond the first nine does not improve any of the provided metrics, including the accuracy score

knn = KNeighborsClassifier()
parameter_grid = {
    "n_neighbors": range(1, 100), #attention!
}

cancer_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10 #attention!
)

parementer_list = KNN_forward_results['feature'][:9].tolist()

cancer_tune_grid.fit(
    X_train[parementer_list],
    Y_train
)

knn_best = KNeighborsClassifier(n_neighbors=cancer_tune_grid.best_params_['n_neighbors'])
knn_best.fit(X_train[parementer_list], Y_train)

# Predict on the test set
X_pred = X_test[parementer_list]
Y_pred = knn_best.predict(X_pred)

print(parementer_list)
print(f"Number of N neighbors in KNN is {cancer_tune_grid.best_params_['n_neighbors']}")
print(f"Accuracy score is {accuracy_score(Y_test, Y_pred)}")
print(f"Precision score is {precision_score(Y_test,Y_pred,pos_label='M')}")
print(f"Recall score is {recall_score(Y_test,Y_pred,pos_label='M')}")
print(f"F1 score is {f1_score(Y_test,Y_pred, average='weighted')}") 


['area_worst', 'compactness_se', 'concavity_mean', 'perimeter_worst', 'area_se', 'compactness_worst', 'concave_points_se', 'smoothness_worst', 'area_mean']
Number of N neighbors in KNN is 5
Accuracy score is 0.993006993006993
Precision score is 0.9818181818181818
Recall score is 1.0
F1 score is 0.9930193167636802
